In [ ]:
import pormake as pm
import numpy as np
import csv
from collections import defaultdict

db = pm.Database()
loc = pm.Locator()

## metal nodes
Fe_oct_a = pm.BuildingBlock(bb_file="pi-d_building_blocks/old_bb/Fe_oct.xyz")
Fe_oct_b = Fe_oct_a.make_chiral_building_block()
Fe_oct = Fe_oct_a

## edges
DHBQ = pm.BuildingBlock(bb_file="pi-d_building_blocks/old_bb/DHBQ.xyz")

pm.log.disable_print()
pm.log.disable_file_print()


## Prepare topologies for screening

In [ ]:
def expand_topo(topo):
    x = 1
    y = 1
    z = 1
    edges = topo.edge_indices
    neigh = topo.neighbor_list
    for i in edges:

        ## below set lists unique neighbors for the edges. If 1, this means expansion is required.
        if len(set([j.index for j in neigh[i]])) == 1:
            dist = [abs(d) for d in neigh[i][0].distance_vector]

            ## expand topology along the "major" edge direction to allow for alternating node sequence.
            if x == 1 and dist.index(max(dist)) == 0:
                x = 2
            if y == 1 and dist.index(max(dist)) == 1:
                y = 2
            if z == 1 and dist.index(max(dist)) == 2:
                z = 2

    return topo * (x, y, z)

In [ ]:
## prep topologies for patterned node assignment

topo_codes = []
file = open('cn34_pid_topology_list.csv')
csvreader = csv.reader(file)

for row in csvreader:
    for topo in row:
        topo_codes.append(topo)

topos = []

for topo in topo_codes:
    cur_topo = db.get_topo(topo)
    cur_topo = expand_topo(cur_topo)

    if cur_topo.n_nodes % 2 == 1:
        ## expand in one of the directions to ensure even number of nodes
        cur_topo = cur_topo * (1, 1, 2)
        
    topos.append(cur_topo)

In [ ]:
## check rmsd and topology scale here (check # nodes again since expansion took place)

node_limit = 80
cn3_topos = []

for topo in topos:
    
    if topo.n_nodes > node_limit:
        continue
    
    if max(topo.unique_cn) == 3:
        cn3_topos.append(topo)

print(len(cn3_topos))

## Perform screening (try different enantionmer arrangements)

In [ ]:

## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=30, check_tetrahedral=False)
successful_cases = []

for topo in cn3_topos:

    ## full single enantiomer considered only once
    single_enantiomer = False    
    
    for chiral_grouping in [1, 2, 3, 4, 6, 8, 12, 16, 81]:

        ## full single enantiomer considered only once
        num_nodes = len(topo.node_indices)        
        if single_enantiomer:
            continue
        if chiral_grouping >= num_nodes:
            single_enantiomer = True
        
        for chiral_begin_flip in [True, False]:

            current_node = {}
            current_edge = {}    

            ## initialize nodes with Fe_oct first (no chirality dependence... is this good?)
            for i, node in enumerate(topo.unique_cn):
                current_node[i] = Fe_oct

            ## initialize edges
            for i, edge in enumerate(topo.unique_edge_types):
                current_edge[tuple(edge)] = DHBQ

            bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
            calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
            calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}

            ## consider chirality of Fe_oct nodes

            chiral_counter = 0
            chiral_flip = chiral_begin_flip

            for node in topo.node_indices:
                
                chiral_counter += 1

                if chiral_flip:
                    bbs[node] = Fe_oct_a
                else:
                    bbs[node] = Fe_oct_b
                    
                if chiral_counter % chiral_grouping == 0:
                    chiral_flip = not chiral_flip


            try:
                mof = test_builder.build(topo, bbs, calc_permutation)

                ## check and eliminate 2D MOFs
                if min(mof.atoms.cell.cellpar()[:3]) < 3:
                    continue                
                    
                mof.view()
                successful_cases.append((topo.name, chiral_grouping, chiral_begin_flip))
                print(topo.name, chiral_grouping, chiral_begin_flip)

            except:
                continue



## Raw screening results

In [ ]:
successful_cases = [('bcu-h', 8, True), #
 ('bpc', 2, True), #
 ('bpc', 2, False), 
 ('eta', 1, True),
 ('eta', 1, False),
 ('eta', 6, True), #
 ('eta-c', 1, True),
 ('eta-c', 1, False),
 ('eta-c', 6, False),
 ('etb', 3, False), #
 ('etc', 1, True), 
 ('etc', 1, False), #
 ('etd', 12, True), #
 ('lig', 4, True),
 ('lig', 4, False), #
 ('noj', 6, True), #
 ('oft', 1, False),
 ('oft', 3, False),
 ('pbz', 12, True),
 ('pbz', 12, False), #
 ('srs', 8, True),
 ('srs', 8, False), #
 ('sxc-d', 12, False), #
 ('ttg', 8, True), #
 ('uni-d', 1, True),
 ('uni-d', 1, False),
 ('uni-d', 12, False),
 ('utg', 8, True),
 ('utg', 8, False)]

## Inspection of raw screening results

Structures from screening are visualized to qualitatively assess their synthesizability. In instances where multiple successful cases are reported for the same topology, case with the smallest residual angle and/or highest symmetry is kept.

In [ ]:

pm.log.enable_print()
pm.log.enable_file_print()

for case in successful_cases:
    
    topo = db.get_topo(case[0])
    chiral_grouping = case[1]
    chiral_begin_flip = case[2]

    current_node = {}
    current_edge = {}    

    ## initialize nodes with Fe_oct first (no chirality dependence... is this good?)
    for i, node in enumerate(topo.unique_cn):
        current_node[i] = Fe_oct

    ## initialize edges
    for i, edge in enumerate(topo.unique_edge_types):
        current_edge[tuple(edge)] = DHBQ

    bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
    calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
    calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}

    ## consider chirality of Fe_oct nodes

    chiral_counter = 0
    chiral_flip = chiral_begin_flip

    for node in topo.node_indices:
        
        chiral_counter += 1

        if chiral_flip:
            bbs[node] = Fe_oct_a
        else:
            bbs[node] = Fe_oct_b
            
        if chiral_counter % chiral_grouping == 0:
            chiral_flip = not chiral_flip


    mof = test_builder.build(topo, bbs, calc_permutation)
    mof.view()
    print(topo.name, chiral_grouping, chiral_begin_flip)



## Final topoologies to be kept

In [ ]:
keep_cases = [successful_cases[i] for i in [0, #bcu-h
                                            1, #bpc
                                            5, #eta
                                            9, #etb
                                            11, #etc
                                            12, #etd
                                            14, #lig
                                            15, #noj
                                            19, #pbz
                                            21, #srs
                                            22, #sxc-d
                                            23, #ttg
                                            26, #uni-d
                                            28, #utg
                                            ]]

In [ ]:
keep_cases = [('bcu-h', 8, True),
 ('bpc', 2, True),
 ('eta', 6, True),
 ('etb', 3, False),
 ('etc', 1, False),
 ('etd', 12, True),
 ('lig', 4, False),
 ('noj', 6, True),
 ('pbz', 12, False),
 ('srs', 8, False),
 ('sxc-d', 12, False),
 ('ttg', 8, True),
 ('uni-d', 12, False),
 ('utg', 8, False)]